In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import os
import json
import pickle
from transformers import BertTokenizer
import torch
import numpy as np
import re

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

In [2]:
model_type = 'cased'

In [3]:
data_path = f"D:/Data/neural-punctuator/szeged_bert_{model_type}/"
file_path = data_path + "szeged.txt"

In [4]:
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.readlines()

In [5]:
len(text), text[1]

(82100,
 ' A szállásunk egy Balaton melletti kis üdülőfaluban, Zamárdiban volt, a Postának az üdülőházában.\n')

In [6]:
from collections import OrderedDict

text = list(OrderedDict.fromkeys(text))
len(text)

80875

In [7]:
from sklearn.utils import shuffle

text = shuffle(text, random_state=0)

text[1]

' Ez jutott az eszébe önkéntelenül.\n'

In [8]:
text = [t.strip() for t in text]
text[1]

'Ez jutott az eszébe önkéntelenül.'

In [9]:
len(' '.join([t.strip() for t in text]).split(' '))

1265372

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(f'bert-base-multilingual-{model_type}')

In [11]:
def clean_text(text):
    text = text.replace('!', '.')
    text = text.replace(':', ',')
    text = text.replace('--', ',')
    text = text.replace('-', ',')
    text = text.replace(';', '.')
    text = text.replace(' ,', ',')
    text = text.replace('♫', '')

    text = re.sub(r'--\s?--', '', text)
    text = re.sub(r'\s+', ' ', text)

    text = text.replace(' ,', ',')
    
    text = re.sub(r',\s?,', ',', text)
    
    return text.strip().lower()

In [12]:
# cleaned_text = [clean_text(t) for t in text]

# for t, tc in zip(' '.join(text).split(' ')[:100], ' '.join(cleaned_text).split(' ')):
#     print(f"{t:20}\t{tc}")

In [13]:
text = [clean_text(t) for t in text]
text[1]

'ez jutott az eszébe önkéntelenül.'

In [14]:
tokenizer.encode(".?,")

[101, 119, 136, 117, 102]

In [15]:
len(' '.join(text).split(' '))

1248861

In [16]:
tokenized = tokenizer.tokenize(' '.join(text))

In [17]:
tokenized[:5]

['az', 'egyes', '_', 'lesz', '##álló']

In [18]:
len(tokenized)

3044475

In [19]:
text2 = tokenizer.convert_tokens_to_string(tokenized)

In [20]:
text2 = [t for t in text2.split(' ') if t not in [',', '.', '?']]
len(text2)

1332371

In [21]:
for t1, t2 in zip(' '.join(text).split(' ')[:1000], text2):
    print(f'{t1:20}{t2}')         

az                  az
egyes_leszállópályánakegyes
például             _
nem                 leszállópályának
ez                  például
volt                nem
a                   ez
neve                volt
akkoriban,          a
angliának           neve
vagy                akkoriban
nagy,britanniának   angliának
nevezték,           vagy
bár                 nagy
londont,            britanniának
ebben               nevezték
meglehetősen        bár
bizonyos            londont
volt                ebben
winston,            meglehetősen
mindig              bizonyos
londonnak           volt
hívták.             winston
ez                  mindig
jutott              londonnak
az                  hívták
eszébe              ez
önkéntelenül.       jutott
könyvtár            az
megosztása          eszébe
közben              önkéntelenül
(                   könyvtár
a                   megosztása
tulajdonságlap      közben
sharing,            (
megosztás           a
fülén               tulajdonsá

gyalog              volna
jött                bele
drezdából           végső
rómába,             soron
a                   fontosnak
via_flaminián       tartom
jött                azt
be,                 hogy
azon                ne
az                  legyek
úton,               egy
amelyről            műveletlen
gimnazista          ember
korában             aki
megtanulta,         csak
hogy                a
ott                 szántáshoz
vonultak            ért
be                  aki
mindig              nem
észak               tudja
felől               hol
a                   van
győzelmes           ecuador
idegenek.           hogy
fordítási           mikor
cselekmények        voltak
jogosságát,         a
azonban             világháborúk
amennyiben          stb
az                  az
ezekből             építészet
származó            történetéről
művek               csak
valós,              annyit
tényleges           lehetett
felhasználása,      megtudni
természetesen       amennyit
az

In [22]:
id2target = {-1: 0,
              119: 1, # .
              136: 2, # ?
              117: 3,  # ,
              -2: -1, # will be masked
             }
target2id = {value: key for key, value in id2target.items()}
    
    
def create_target(encoded):
    targets = []
    text = []

    target = -2 # Always mask after [CLS] token

    text.append(encoded[0])
    idx = 1
    while idx < len(encoded):
        word = encoded[idx]
        if word in id2target.keys():
            if word in (119,117) and \
            tokenizer._convert_id_to_token(encoded[idx-1]).lstrip('▁').isnumeric() and \
            tokenizer._convert_id_to_token(encoded[idx+1]).startswith('##') and \
            tokenizer._convert_id_to_token(encoded[idx+1]).strip('#').isnumeric():
                targets.append(-2) # prev number
#                 print(-2)
#                 print(f"{tokenizer._convert_id_to_token(word):15}\t", end="")
                text.append(word)                
                target = -2 # '. or ,'
            else:
                target = word
        else:
            if tokenizer._convert_id_to_token(word).startswith('##'):
                target = -2
            targets.append(target)
#             print(target)
#             print(tokenizer._convert_id_to_token(word), '\t', end="")
            
            target = -1
            text.append(word)
            
        idx += 1

    targets.append(target)

    targets = [id2target[t] for t in targets]

    return text, targets

In [23]:
train_n = 70_000
valid_n = 8_000

train_text = ' '.join(text[:train_n])
valid_text = ' '.join(text[train_n:train_n+valid_n])
test_text = ' '.join(text[train_n+valid_n:])

len(train_text.split(' ')), len(valid_text.split(' ')), len(test_text.split(' '))

(1080644, 123975, 44242)

In [24]:
len(text) - train_n - valid_n

2875

In [25]:
train_tokens = tokenizer.encode(train_text)
valid_tokens = tokenizer.encode(valid_text)
test_tokens = tokenizer.encode(test_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (2635119 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (301868 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (107494 > 512). Running this sequence through the model will result in indexing errors


In [26]:
train_tokens, train_targets = create_target(train_tokens)
valid_tokens, valid_targets = create_target(valid_tokens)
test_tokens, test_targets = create_target(test_tokens)

In [27]:
# For backward campatibility
train_tokens, train_targets = [train_tokens], [train_targets]
valid_tokens, valid_targets = [valid_tokens], [valid_targets]
test_tokens, test_targets = [test_tokens], [test_targets]

In [28]:
with open(data_path + 'train_data.pkl', 'wb') as f:
    pickle.dump((train_tokens, train_targets), f)
with open(data_path + 'valid_data.pkl', 'wb') as f:
    pickle.dump((valid_tokens, valid_targets), f)
with open(data_path + 'test_data.pkl', 'wb') as f:
    pickle.dump((test_tokens, test_targets), f)

In [29]:
(np.array(train_targets) >= 0).sum()

1153158

In [30]:
(np.array(train_targets) >= 0).sum(), (np.array(valid_targets) >= 0).sum(), (np.array(test_targets) >= 0).sum()

(1153158, 132224, 46992)

In [31]:
sum(((np.array(train_targets) >= 0).sum(), (np.array(valid_targets) >= 0).sum(), (np.array(test_targets) >= 0).sum()))

1332374

In [32]:
from collections import Counter

for tr in (train_targets, valid_targets, test_targets):
    c = Counter((t for targets in tr for t in targets))
    print('\t'.join([str(c[i]) for i in (1,2,3,0,-1)]))

81135	1681	126929	943413	1264300
9207	178	14505	108334	144828
3369	70	5141	38412	51591


In [33]:
sum((np.array(vt) != -1).sum() for vt in train_targets), \
sum((np.array(vt) != -1).sum() for vt in valid_targets), \
sum((np.array(vt) != -1).sum() for vt in test_targets)

(1153158, 132224, 46992)

In [34]:
sum((np.array(vt) == 2).sum() for vt in test_targets)

70

In [35]:
sum(len(vt) for vt in train_targets), \
sum(len(vt) for vt in valid_targets), \
sum(len(vt) for vt in test_targets)

(2417458, 277052, 98583)

In [36]:
word_idx = 0
s = []
for te, ta in zip(test_tokens[0][:], test_targets[0]):
#     print(tokenizer._convert_id_to_token(te), ta)
    if ta != -1:
        s.append(te)
        decoded = tokenizer.decode(s)
        print(f"{word_idx:3}. {decoded:20}\t{ta}\t{[tokenizer._convert_id_to_token(ss) for ss in s]}")
        word_idx += 1
        s = []
    else:
        s.append(te)

  0. [CLS] hála          	0	['[CLS]', 'há', '##la']
  1. az                  	0	['az']
  2. úrnak               	3	['úr', '##nak']
  3. nagyon              	0	['nagyon']
  4. jó                  	0	['jó']
  5. képességeim         	0	['képes', '##sége', '##im']
  6. vannak              	3	['vannak']
  7. és                  	0	['és']
  8. nagyon              	0	['nagyon']
  9. szívesen            	0	['sz', '##ív', '##esen']
 10. tanítanék           	0	['tan', '##ít', '##ané', '##k']
 11. egyetemeken         	3	['egy', '##etem', '##eken']
 12. főiskolákon         	0	['fő', '##isko', '##lá', '##kon']
 13. is                  	0	['is']
 14. teológusokat        	3	['te', '##ol', '##óg', '##usok', '##at']
 15. filozófusokat       	1	['filo', '##zó', '##fus', '##okat']
 16. ha                  	0	['ha']
 17. az                  	0	['az']
 18. ember               	0	['ember']
 19. mindig              	0	['mindig']
 20. úgy                 	0	['úgy']
 21. gondolkozna         	3	['go', '##ndo', 

697. győrött             	0	['g', '##y', '##őr', '##ött']
698. még                 	0	['még']
699. 1998                	0	['1998']
700. márciusában         	0	['március', '##ában']
701. száz                	0	['sz', '##áz']
702. százalékkal         	0	['sz', '##ázal', '##ék', '##kal']
703. megemelték          	0	['meg', '##eme', '##lték']
704. a                   	0	['a']
705. bérleti             	0	['bé', '##rle', '##ti']
706. díjakat             	1	['díj', '##akat']
707. az                  	0	['az']
708. igazgatóság         	0	['iga', '##z', '##gató', '##ság']
709. december            	0	['december']
710. 10                  	3	['10']
711. én                  	0	['én']
712. felhatalmazta       	0	['fel', '##hat', '##al', '##maz', '##ta']
713. az                  	0	['az']
714. ápv                 	0	['áp', '##v']
715. _                   	0	['_']
716. rt                  	1	['r', '##t']
717. elnök               	3	['el', '##nök']
718. vezérigazgatóját    	0	['vez', '##éri', '##ga', 

1378. kívül               	1	['kívül']
1379. a                   	0	['a']
1380. suli                	0	['sul', '##i']
1381. előtt               	0	['előtt']
1382. gyülekezett         	0	['g', '##yü', '##lek', '##ezett']
1383. az                  	0	['az']
1384. egész               	0	['egész']
1385. osztály             	3	['os', '##zt', '##ály']
1386. hogy                	0	['hogy']
1387. elinduljunk         	0	['eli', '##ndu', '##lju', '##nk']
1388. a                   	0	['a']
1389. cél                 	0	['c', '##él']
1390. felé                	1	['felé']
1391. miután              	0	['miután']
1392. a                   	0	['a']
1393. lány                	0	['lá', '##ny']
1394. kiment              	3	['kim', '##ent']
1395. egy                 	0	['egy']
1396. ideig               	0	['ideig']
1397. csend               	0	['c', '##send']
1398. volt                	1	['volt']
1399. hát                 	0	['hát']
1400. ilyen               	0	['ilyen']
1401. még                 	0	['még'

2129. a                   	0	['a']
2130. részvényárak        	3	['rész', '##vén', '##y', '##ára', '##k']
2131. a                   	0	['a']
2132. fő                  	0	['fő']
2133. húzóerőt            	0	['h', '##ú', '##zó', '##er', '##őt']
2134. a                   	0	['a']
2135. technológiai        	0	['tech', '##nol', '##ógiai']
2136. részvények          	0	['rész', '##vén', '##yek']
2137. és                  	0	['és']
2138. pénzügyi            	0	['pé', '##nz', '##ügyi']
2139. szektor             	0	['sz', '##ektor']
2140. papírjai            	0	['pa', '##pí', '##rja', '##i']
2141. adták               	1	['adták']
2142. szeretnék           	0	['sz', '##eret', '##né', '##k']
2143. önállóan            	0	['önálló', '##an']
2144. élni                	1	['él', '##ni']
2145. kétszíjas           	1	['két', '##sz', '##í', '##jas']
2146. maradj              	0	['mara', '##d', '##j']
2147. állva               	3	['áll', '##va']
2148. ott                 	3	['ott']
2149. ahol               

2767. baj                 	0	['ba', '##j']
2768. mégis               	0	['mégis']
2769. nagy                	3	['nagy']
2770. egy                 	0	['egy']
2771. belső               	0	['belső']
2772. vizsgálat           	3	['viz', '##sg', '##ála', '##t']
2773. amelyet             	0	['amelyet']
2774. az                  	0	['az']
2775. özvegy              	3	['öz', '##veg', '##y']
2776. gergely             	0	['ger', '##gel', '##y']
2777. _                   	0	['_']
2778. irén                	0	['ir', '##én']
2779. kért                	0	['k', '##ért']
2780. a                   	0	['a']
2781. p                   	1	['p']
2782. _                   	0	['_']
2783. m                   	1	['m']
2784. _                   	0	['_']
2785. s                   	1	['s']
2786. _                   	0	['_']
2787. kft                 	3	['k', '##ft']
2788. től                 	3	['től']
2789. azt                 	0	['azt']
2790. állapította         	0	['áll', '##ap', '##ította']
2791. meg          

...

43777. a                   	0	['a']
43778. jegyzési            	0	['jeg', '##y', '##zési']
43779. ív                  	0	['í', '##v']
43780. tartalmára          	0	['tar', '##tal', '##má', '##ra']
43781. vonatkozóan         	0	['von', '##at', '##kozó', '##an']
43782. is                  	0	['is']
43783. ad                  	0	['ad']
43784. útmutatatást        	0	['út', '##mut', '##ata', '##tás', '##t']
43785. a                   	0	['a']
43786. cg                  	1	['c', '##g']
43787. _                   	0	['_']
43788. törv                	1	['t', '##ör', '##v']
43789. ii                  	1	['ii']
43790. 32                  	1	['32']
43791. 968                 	0	['968']
43792. /                   	0	['/']
43793. 1991                	0	['1991']
43794. /                   	0	['/']
43795. 4                   	1	['4']
43796. számú               	0	['számú']
43797. határozat           	0	['hat', '##ár', '##oza', '##t']
43798. leszögezve          	3	['lesz', '##ög', '##ez', '##ve']
4379

44617. is                  	0	['is']
44618. csak                	0	['csak']
44619. feltétlen           	0	['felt', '##ét', '##len']
44620. lehet               	0	['lehet']
44621. és                  	0	['és']
44622. az                  	0	['az']
44623. esetleges           	0	['ese', '##tle', '##ges']
44624. feltételt           	0	['felt', '##ét', '##elt']
44625. nem                 	0	['nem']
44626. létezőnek           	0	['l', '##ét', '##ező', '##nek']
44627. kell                	0	['kell']
44628. tekinteni           	1	['teki', '##nten', '##i']
44629. persze              	0	['pers', '##ze']
44630. ezt                 	0	['ezt']
44631. csak                	0	['csak']
44632. azért               	0	['azért']
44633. sikerült            	0	['sikerült']
44634. elérnem             	3	['el', '##ér', '##nem']
44635. mert                	0	['mert']
44636. meg                 	0	['meg']
44637. kellett             	0	['kellett']
44638. állni               	0	['áll', '##ni']
44639. tankolni      

45421. azt                 	0	['azt']
45422. érhetik             	0	['é', '##r', '##het', '##ik']
45423. el                  	0	['el']
45424. vele                	3	['vele']
45425. hogy                	0	['hogy']
45426. programjuk          	0	['program', '##juk']
45427. 4                   	0	['4']
45428. gigabájt            	0	['gi', '##gab', '##á', '##jt']
45429. memóriánál          	0	['me', '##mó', '##rián', '##ál']
45430. többet              	0	['több', '##et']
45431. is                  	0	['is']
45432. elérhessen          	3	['el', '##ér', '##hessen']
45433. s                   	0	['s']
45434. ez                  	0	['ez']
45435. fontos              	0	['fontos']
45436. például             	0	['például']
45437. nagy                	0	['nagy']
45438. adatbázisok         	0	['adat', '##bá', '##zis', '##ok']
45439. esetén              	3	['esetén']
45440. mert                	0	['mert']
45441. azoknak             	3	['azok', '##nak']
45442. ha                  	0	['ha']
45443. a   

46177. eddigi              	0	['ed', '##dig', '##i']
46178. ismeretek           	0	['is', '##mere', '##tek']
46179. alapján             	0	['alapján']
46180. együttesen          	0	['együttes', '##en']
46181. csupán              	0	['csupán']
46182. 6                   	3	['6']
46183. 7                   	0	['7']
46184. ezer                	0	['ezer']
46185. részvényes          	0	['rész', '##vén', '##yes']
46186. és                  	0	['és']
46187. kötvényes           	0	['k', '##öt', '##vén', '##yes']
46188. tett                	0	['tett']
46189. így                 	0	['így']
46190. a                   	0	['a']
46191. körülbelül          	0	['körülbelül']
46192. 25                  	3	['25']
46193. 30                  	0	['30']
46194. ezerből             	3	['ezer', '##ből']
46195. a                   	0	['a']
46196. többiek             	0	['többi', '##ek']
46197. elsősorban          	0	['elsősorban']
46198. azért               	0	['azért']
46199. nem                 	3	['nem']
462

46981. az                  	0	['az']
46982. ország              	0	['ország']
46983. elindult            	0	['eli', '##ndu', '##lt']
46984. a                   	0	['a']
46985. nyugati             	0	['nyugati']
46986. modernizáció        	0	['moderni', '##zá', '##ció']
46987. felé                	1	['felé']
46988. te                  	0	['te']
46989. jó                  	0	['jó']
46990. isten               	1	['iste', '##n']
46991. [SEP]               	0	['[SEP]']
